In [15]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as file_handle:
    text = file_handle.read()

chars = sorted(set(text))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [16]:
str_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_ch = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_ch[n] for n in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [17]:
import torch
from torch import nn
from torch.nn import functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BLOCK_SIZE = 8
BATCH_SIZE = 4

In [18]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36,
        11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32,
        39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39,
        30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1,
        39, 50,  9,  1, 39, 50, 37, 25,  1, 39])


In [19]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))
    print(ix)
    x = torch.stack([data[i:i+BLOCK_SIZE]for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1]for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch(split="train")
print(f'Inputs: {x} targets: {y}')

tensor([103697, 122412, 104053,  57720])
Inputs: tensor([[ 1, 73, 61, 58, 71, 58,  1, 76],
        [67, 78,  1, 73, 78, 69, 58, 72],
        [60, 71, 68, 76, 65, 72,  1, 73],
        [73,  1, 55, 68, 73, 61, 58, 71]], device='cuda:0') targets: tensor([[73, 61, 58, 71, 58,  1, 76, 54],
        [78,  1, 73, 78, 69, 58, 72,  9],
        [71, 68, 76, 65, 72,  1, 73, 68],
        [ 1, 55, 68, 73, 61, 58, 71,  1]], device='cuda:0')


In [20]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]

for i in range(BLOCK_SIZE):
    context = x[:i+1]
    target = y[i]
    print(f'When input is {context}, target is {target}')

When input is tensor([80]), target is 28
When input is tensor([80, 28]), target is 39
When input is tensor([80, 28, 39]), target is 42
When input is tensor([80, 28, 39, 42]), target is 39
When input is tensor([80, 28, 39, 42, 39]), target is 44
When input is tensor([80, 28, 39, 42, 39, 44]), target is 32
When input is tensor([80, 28, 39, 42, 39, 44, 32]), target is 49
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]), target is 1


In [26]:
from torch import nn

vocab_size = len(chars)
embedding_size = 100
embedding = nn.Embedding(vocab_size, embedding_size)

input_indicies = torch.LongTensor([1, 5, 3, 2])
output = embedding(input_indicies)

print(vocab_size)
output.shape

81


torch.Size([4, 100])

In [28]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else: 
            loss = None
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(index)
            # focus on last time step
            logits = logits[:,-1,:] # (B, C)
            
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size=vocab_size)
model = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = model.generate(context, max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)

        
    


L'U.a(us;Jtf)qVuHJvhf)?nM-.3"6N :4JEpQJ*GAOmE:a;7&n0﻿CrWn"6bdh8tQc_pMlyE20Uh8h:9f5aJEp[0]J;k(V7layB?ah﻿r'zZyP
aTT:::rS;(U_DzkhkG4Jy﻿xQyPz0UQnMun9-&NCfhglqpm[MrcKC?DpPiJsgq.82Hf'U9l8ycnFtb﻿CWoDSQt!ap&E?aDWv"-,&pt H﻿xoFUoH.9;A]hs-?a74jMIM&52'9Jf)
vu8TQs
,WfV*41g]6-lweS;*SQ'*yP﻿'rK4A1hX5UK﻿
l'd:pc)Sas
(L238S50pDrT:SkW)4CZB]f1W& DB&Q"4"R"﻿24Jv_ZF'k'8Qtf7xIFK*mn﻿CTEao-mB]zM5n,ps-G3_yU?oOdNWbh,urg
_AlbXM53)W]47P
2N*a]z8eb*Dtq;L?DSUhsHuZ07ab'.f0uv0xl'b;LV,;Ard!8[﻿C(wvuh&y'z0sm?7kTPi)jfXtg [﻿MI,.9;*aEl&
